<a href="https://colab.research.google.com/github/megonzau/covidcol/blob/main/Covid_19_in_Colombia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import scipy.io as spio
from functools import reduce
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.optimize import minimize


# COVID-19 Community Mobility - Google 


In [ ]:
df_google = pd.read_csv('https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv', delimiter=",", decimal=".") #Datos de Google

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
#Filtrar datos de Colombia
df_g_col0 = df_google[ df_google["country_region"] == "Colombia" ]
#Llenar los espacios vacios de sub_region_1 ( los espacios vacios son los datos a nivel nacional)
df_g_col0["sub_region_1"].fillna("Nacional", inplace = True) 
df_g_col=df_g_col0[ df_g_col0["sub_region_1"] == "Nacional" ]
pd.to_datetime(df_g_col['date'], format= "%Y-%m-%d")
df_g_col.drop(columns=['country_region_code', 'country_region',"sub_region_2","metro_area","iso_3166_2_code","census_fips_code"], inplace = True)
df_g_col.set_index(['date'],inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Casos COVID-19 

In [ ]:
#Base de datos
df_covid = pd.read_csv('https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv?accessType=DOWNLOAD', delimiter=",", decimal=".") #Datos de COVID por INS


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
#Renombar las columnas y manejo ciertas varibales
df_covid.rename(columns={"Fecha de muerte":'dateM'}, inplace=True)
df_covid[["dateMuerte","timeEl"]]=df_covid.dateM.str.split(" ",expand=True) #INS coloca la fecha y la hora, se separa la fecha

df_covid.rename(columns={"fecha reporte web":'date0'}, inplace=True)
df_covid[["date","timeE"]]=df_covid.date0.str.split(" ",expand=True) #INS coloca la fecha y la hora, se separa la fecha

df_covid.rename(columns={"Fecha de recuperación":'dateR'}, inplace=True)
df_covid[["dateR","timeE"]]=df_covid.dateR.str.split(" ",expand=True) #INS coloca la fecha y la hora, se separa la fecha

df_covid["casos"] = 1

#Casos por fecha 
covid_dep0= df_covid.groupby([ "date"])['casos'].sum() 
df_covid_dep=covid_dep0.reset_index()

#Fallecidos por fecha
df_covid2 = df_covid[df_covid.dateMuerte.notnull()]
covid_depM= df_covid2.groupby(["dateMuerte"])['casos'].sum() 
df_covid_depM=covid_depM.reset_index()

#Recuperados
df_covid3 = df_covid[df_covid.dateR.notnull()]
df_covid_depR= df_covid3.groupby(["dateR"])['casos'].sum() 
df_covid_depR=df_covid_depR.reset_index()

In [ ]:
#Fallecidos
df_covid_depM2 = df_covid_depM.rename(columns = {"dateMuerte": "date",
                                                "casos":"fallecidos"}) 
df_covid_depM2=df_covid_depM2.set_index(['date'])
#Casos confirmados
df_covid_dep2 = df_covid_dep.rename(columns = {"casos": "confirmados"}) 
df_covid_dep2=df_covid_dep2.set_index([ 'date'])
#Recuperados
df_covid_depR=df_covid_depR.rename(columns={
                              "dateR":'date',
                                "casos":'Recuperados'})
df_covid_depR=df_covid_depR.set_index([ 'date'])

#Merge casos, fallecidos y recuperados
df_0= pd.merge(df_covid_depM2,df_covid_dep2, how='outer', left_index=True, right_index=True)
df_depcovid0= pd.merge(df_covid_depR,df_0, how='outer', left_index=True, right_index=True)

df_depcovid0=df_depcovid0.reset_index()
df_depcovid0['date']=  pd.to_datetime(df_depcovid0['date'], format= "%d/%m/%Y")
df_depcovid1=df_depcovid0.set_index(['date'])

#Merge con Google

df_depcovid= pd.merge(df_g_col,df_depcovid1, how='outer', left_index=True, right_index=True)
df_depcovid=df_depcovid.reset_index().fillna(0)

#Ordenar 
df_depcovid.sort_values(by=['date'], inplace=True, ascending=True)

In [ ]:
df_depcovid

,date,sub_region_1,place_id,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,Recuperados,fallecidos,confirmados
0,2020-02-15,Nacional,ChIJo5QVrjqkFY4RQKPy7wSaDZo,4.0,5.0,-5.0,1.0,7.0,1.0,0.0,0.0,0.0
1,2020-02-16,Nacional,ChIJo5QVrjqkFY4RQKPy7wSaDZo,0.0,5.0,-5.0,-1.0,1.0,2.0,0.0,0.0,0.0
2,2020-02-17,Nacional,ChIJo5QVrjqkFY4RQKPy7wSaDZo,-1.0,3.0,-4.0,6.0,15.0,-1.0,0.0,0.0,0.0
3,2020-02-18,Nacional,ChIJo5QVrjqkFY4RQKPy7wSaDZo,-3.0,1.0,-3.0,10.0,17.0,-1.0,0.0,0.0,0.0
4,2020-02-19,Nacional,ChIJo5QVrjqkFY4RQKPy7wSaDZo,-5.0,-5.0,-5.0,6.0,13.0,-2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
397,2021-03-18,Nacional,ChIJo5QVrjqkFY4RQKPy7wSaDZo,-28.0,9.0,-27.0,-17.0,-18.0,10.0,4519.0,123.0,5139.0
398,2021-03-19,Nacional,ChIJo5QVrjqkFY4RQKPy7wSaDZo,-23.0,15.0,-23.0,-10.0,-11.0,9.0,4770.0,131.0,5133.0
399,2021-03-20,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4604.0,99.0,6761.0
400,2021-03-21,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4705.0,112.0,5963.0


In [ ]:
df_depcovid=df_depcovid[ df_depcovid["sub_region_1"] == "Nacional" ]
df_depcovid.set_index(['sub_region_1',"date"], inplace= True)
df_depcovid= pd.concat([df_depcovid, df_depcovid.groupby(level=0).cumsum().add_prefix('Acum_')], 1)

In [ ]:
df_depcovid=df_depcovid.reset_index().set_index(["date"])

In [ ]:
df_depcovid=df_depcovid.drop(columns=['Acum_retail_and_recreation_percent_change_from_baseline', 'Acum_grocery_and_pharmacy_percent_change_from_baseline',"Acum_parks_percent_change_from_baseline","Acum_transit_stations_percent_change_from_baseline","Acum_workplaces_percent_change_from_baseline","Acum_residential_percent_change_from_baseline"])

# MAtrices - MODULO SIR

Working process. 

In [ ]:
df_depcovid=df_depcovid.reset_index()
infected= df_depcovid[['Acum_confirmados','date']].set_index(["date"])
infected=infected.T
infected["population"]=50372424
infected["Tipo"]="Nacional"
infected=infected.reset_index().set_index(["Tipo"])

In [ ]:
deaths= df_depcovid[['Acum_fallecidos','date']].set_index(["date"])
deaths=deaths.T
deaths["population"]=50372424
deaths["Tipo"]="Nacional"
deaths=deaths.reset_index().set_index(["Tipo"])

In [ ]:
recovered= df_depcovid[['Acum_Recuperados','date']].set_index(["date"])
recovered=recovered.T
recovered["population"]=50372424
recovered["Tipo"]="Nacional"
recovered=recovered.reset_index().set_index(["Tipo"])